# Reactive Programming Tutorial
### With elements from functional programming

In this tutorial we will be using reactive programming to process data emitted from a game. 
#### Brief Overview
- We will start by processing a stream of the players who want to play the game, filtering out usernames that are unallowed.
- Next we will assign each player to one of two teams
- Then we will process data from the first round of the game
- Then process data from the final round of the game

First, we must import the reactivex module in order to easily program in a reactive style in Python.

If you do not have reactivex installed in your current Python environment, you can install it with the following command in the terminal:

    pip install reactivex

Once reactivex is installed, import it into your project.

In [1]:
import reactivex as rx
import reactivex.operators as op

Next, we will create an Observable using the factory method 

    rx.of()

This method takes any number of arguments and returns an Observable that will output the arguments in a stream. We will use this to simulate a stream of usernames of peiple who want to play the game. In a real world application this list would most likely not be known in advance, and instead other reactive programming methods would be used to receive the stream, likely over a network.

We will also declare a set of names that are prohibited.

In [2]:
names = rx.of("Albert", "Edna", "Gertrude", "Hannah", "Elle", "Anna", "Booty", "Samantha", "Cornelia", "Sherman", "Winston")
prohibited_names = {"Booty", "Bot"}

One of the requirements of the game is that usernames are not alowed to be palindromes. In order to test a name to determine if it is a palindrome, we will define a boolean method in a 'Functional Programming' style. The significant features fo functional programming we will use in this method will be: treating variables as if they are immutable, and using recursion instead of iteration.

Observe that the following example is NOT written in a functional style:

In [3]:
def nonFunctional_isPalindrome(s):

    if len(s) <= 1:
        return True
    
    s = s.lower()

    l_index = 0
    r_index = len(s) - 1

    while l_index < r_index:

        print(s[l_index] + " " + s[r_index])

        if s[l_index] != s[r_index]:
            return False
        else:
            l_index += 1
            r_index -= 1

    return True
    

Why does the previous code break functional programming conventions?

**Write Answer Here**

Now, write a new Palindrome function that follows functional programming conventions.

In [4]:
"""Functional Style method to identify palindromes"""
def functional_isPalindrome(s):

    if len(s) <= 1:
        return True
    
    new_s = s.lower()
    
    if (new_s[0] == new_s[-1]):
        return functional_isPalindrome(new_s[1:len(new_s)-1])
    
    else:
        return False

Now we have two functions (nonFunctional_isPalindrome and functional_isPalindrome) that can identify whether a string is a palindrome or not.  Perhaps not with this particular example, but sometimes, we may prefer to use one function over another in certain situations.  It is here that we can take advantage of another aspect of functional programming, first-class and higher-order functions.

In the following block, implement a function "isPalindrome(), that takes advantage of the principle of first class and higher order functions in order to allow for the passing of function that they want to run (nonFunctional_isPalindrome, or functional_isPalindrome). If no specific method is specified, the function should use the functional_isPanlindrome method.



In [5]:
def isPalindrome(n, foo = functional_isPalindrome):

    return foo(n)

In [6]:
player_stream = names.pipe(
    op.filter(lambda n : (n not in prohibited_names) and (not isPalindrome(n)))
)
# After filtering, player_stream should have 6 items

### Playing the game

The first step in playing the game is assigning the players to a team. We will accomplish this by first creating 2 sets, one that represents the members of the red team, and one that represents the members of the blue team. Players will be assigned to the team in alternating order, starting with the red team. If there is an odd number of players, a player named "Bot" will be added to the blue team to even out the ranks (we will also need to add Bot to the end of the player stream).

In [7]:
red_team = set()
blue_team = set()

total_players = 0

def add_player(player, team):
    global total_players 
    total_players += 1
    team.add(player)

# def add_bot():
#     blue_team.add("Bot")


player_stream.subscribe(
    lambda name : add_player(name, red_team) if (not total_players % 2) else add_player(name, blue_team)
)

if (total_players % 2):
    add_player("Bot", blue_team)
    player_stream = rx.concat(player_stream, rx.of("Bot"))

player_stream.subscribe(
    lambda x : print(x)
)

print(red_team)
print(blue_team)

Albert
Edna
Gertrude
Samantha
Cornelia
Sherman
Winston
Bot
{'Gertrude', 'Winston', 'Albert', 'Cornelia'}
{'Samantha', 'Edna', 'Sherman', 'Bot'}


In order to simulate "different" games being played, we will use a random number generator to determine how many points each player gets. We will create a stream of point values, that we will combine with the stream of player names in order to generate a stream of tuples that contain the name and number of points for each player. Using this stream, we will update the point values for each team.

In [8]:
from random import randint

In [9]:
r1_point_stream = rx.from_iterable([randint(0,20) for x in range(total_players)])

scores = rx.zip(player_stream, r1_point_stream)   # Creates a stream of tuples in the form (player_name, score)

point_totals = [0,0]    # [red team score, blue team score]

def update_points(score_tuple):
    if score_tuple[0] in red_team:
        point_totals[0] += score_tuple[1]
    else:
        point_totals[1] += score_tuple[1]

scores.subscribe(
    lambda score_tuple : update_points(score_tuple)
)

print(point_totals)

[19, 14]


### Final Round

In the final round, there will be some modifiers applied to the individaul scores. We will acheive this by mapping values that meet certain criteria.

In [15]:
# Create a new stream of point values

final_point_stream = rx.from_iterable([randint(0,20) for x in range(total_players)])

# This time we'll be doing some transformations on the point values
    # if the score is divisible by 2, The score is doubled
    # if the score is divisible by 3, the score is divided by 3
    # if the player is on the losing team, their score is multiplied by 1.1 (if the score is a tie, the red team will be considered the losing team)
# Note: the first 2 conditions can be applied directly to the point stream, but the third one requires us to know the players associated with each score

losing_team = red_team
if point_totals[1] < point_totals[0]:
    losing_team = blue_team

final_point_stream.pipe(
    op.map(lambda x : 2 * x if (not x % 2) else x),
    op.map(lambda x : x / 3.0 if (not x % 3) else x)
)

f_scores = rx.zip(player_stream, final_point_stream)

def final_update_points(score_tuple):
    if score_tuple[0] in losing_team:
        score_tuple = (score_tuple[0], 1.1 * score_tuple[1])
    update_points(score_tuple)

def print_final_results():
    if point_totals[0] > point_totals[1]:
        print(f"The winner is the red team with a score of {point_totals[0]} to {point_totals[1]}")
    else:
        print(f"The winner is the blue team with a score of {point_totals[1]} to {point_totals[0]}")

f_scores.subscribe(
    on_next=lambda score_tuple : final_update_points(score_tuple),
    on_completed= lambda : print_final_results()
)

The winner is the red team with a score of 203.7 to 169.5
